In [2]:
import numpy as np
import pandas as pd
import os
from pathlib import Path

In [3]:
data_path = Path('c:/Users/257805/uploadGit/titanic_practice',"titanic")

In [4]:
# data_path内に読み込みたいファイルあるか確認．
files = os.listdir(data_path)
# ファイル名の一覧を表示
print(files)

['gender_submission.csv', 'test.csv', 'train.csv']


In [105]:
df_gender = pd.read_csv(Path(data_path, files[0])) # 自作モデルの完成度check
df_test = pd.read_csv(Path(data_path, files[1])) # 自作モデルの出力file
df_train = pd.read_csv(Path(data_path, files[2])) # 学習file

In [106]:
# 特徴量エンジニアリング：読み込んだデータをMLアルゴが扱える形に変換・読み込んだデータから新たな特徴量を作成
data = pd.concat([df_train, df_test], sort=False) # df_trainとdf_testを結合

In [107]:
# DataFrame内にNaNが含まれているかどうかを確認
has_nans = pd.DataFrame([data[i].isnull().any() for i in data.columns], index = data.columns, columns = ["Nan"])
has_nans

,Nan
PassengerId,False
Survived,True
Pclass,False
Name,False
Sex,False
Age,True
SibSp,False
Parch,False
Ticket,False
Fare,True


In [110]:
# 上結果より，Survived, Age, Fare, Cabiin, Embarkedに欠損値あることがわかる．⇒調整の必要あり
# inplace = Falseとすること．(元データが変更されるのを避けるため)
data_replace = data.copy()
data_replace["Sex"]=data_replace["Sex"].replace(["male", "female"],[0,1]).astype(int) # 性別をmale = 0, female = 1に変換する
data_replace["Fare"]=data_replace["Fare"].fillna(np.mean(data["Fare"])) # coulumn"Fare"のNanを平均値に置き換え
data_replace['Embarked']=data_replace['Embarked'].fillna(('S')) # Embarkedの欠損地をSに置き換える．
data_replace['Embarked'] = data_replace['Embarked'].map({'S': 0, 'C': 1, 'Q': 2}).astype(int) # S,C,Vを0,1,2にうつす？
data_replace['Fare'] = data_replace['Fare'].fillna(np.mean(data['Fare'])) # Fareの欠損値を平均値で埋める
age_avg = data_replace['Age'].mean() # ageの欠損値を平均値の1sigma周りの乱数で埋める．
age_std = data_replace['Age'].std()
data_replace['Age'] = data_replace['Age'].fillna(np.random.randint(age_avg - age_std, age_avg + age_std))

C:\Users\257805\AppData\Local\Temp\ipykernel_21600\1143559666.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_replace["Sex"]=data_replace["Sex"].replace(["male", "female"],[0,1]).astype(int) # 性別をmale = 0, female = 1に変換する


In [123]:
delete_columns = ['Name', 'PassengerId', 'SibSp', 'Parch', 'Ticket', 'Cabin']
data_replace = data_replace.drop(delete_columns, axis=1) #不要なデータ列削除

In [124]:
df_train = data_replace[:len(df_train)]
df_test = data_replace[len(df_train):]
y_train = (df_train.copy())['Survived']
X_train = (df_train.copy()).drop('Survived', axis=1)
X_test = (df_test.copy()).drop('Survived', axis=1)

In [133]:
X_train.head()

,Pclass,Sex,Age,Fare,Embarked
0,3,0,22.0,7.2500,0
1,1,1,38.0,71.2833,1
2,3,1,26.0,7.9250,0
3,1,1,35.0,53.1000,0
4,3,0,35.0,8.0500,0


In [126]:
# 用意した特徴量と予測の対象ペアからMLアルゴを用いて対応関係を学習させる：
from sklearn.linear_model import LogisticRegression

In [134]:
clf = LogisticRegression(penalty='l2', solver='sag', random_state=0)

In [135]:
clf.fit(X_train, y_train)

c:\Users\257805\rye_folder\rye_kaggle\.venv\lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


LogisticRegression(random_state=0, solver='sag')

In [136]:
y_pred = clf.predict(X_test)

In [137]:
y_pred[:20]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0.])

In [ ]:
sub = pd.read_csv()
sub['Survived'] = list(map(int, y_pred))
sub.to_csv('submission.csv', index=False)

In [139]:
df_gender["Survived"] = list(map(int,y_pred))
df_gender.to_csv("submission.csv", index = False)